**Model Health Dashboard**


This dashboard reports statistics about the health of the collection of Drawdown solution models.

In [1]:
import importlib
import os.path

import bqplot
import IPython.display
import ipywidgets
import pandas as pd

import solution.factory

all_solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'), index_col=False,
                            skipinitialspace=True, header=0, skip_blank_lines=True, comment='#')
total_solutions = len(all_solutions.index)
py_solutions_scenarios = solution.factory.all_solutions_scenarios()

num_py_solutions = len(list(py_solutions_scenarios.keys()))
num_xl_solutions = total_solutions - num_py_solutions

pds_adoption_basis_counts = {
    'Linear': 0,
    'Existing Adoption Prognostications': 0,
    'S-Curve': 0,
    'Fully Customized PDS': 0,
    'Customized S-Curve Adoption': 0,
}
ref_adoption_basis_counts = {
    'Default': 0,
    'Custom': 0,
}
for name in py_solutions_scenarios.keys():
    m = importlib.import_module('solution.'+name)
    for scenario in m.scenarios.values():
        pds_adoption_basis_counts[scenario.soln_pds_adoption_basis] += 1
        ref_basis = scenario.soln_ref_adoption_basis
        ref_basis = ref_basis if ref_basis is not None else 'Default'
        ref_adoption_basis_counts[ref_basis] += 1

soln_count_chart = bqplot.Pie(sizes=[num_xl_solutions, num_py_solutions], labels=['Excel only', 'Python & Excel'],
                              display_values=True, values_format='d', display_labels='inside', radius=110)
soln_count_fig = bqplot.Figure(marks=[soln_count_chart], title='Solution implementation')
soln_count_fig.layout.width = '300px'
soln_count_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}
pds_adoption_chart = bqplot.Pie(sizes=list(pds_adoption_basis_counts.values()),
                                labels=list(pds_adoption_basis_counts.keys()),
                                display_labels='outside', radius=140,
                                sort=False, display_values=True, values_format='d')
pds_adoption_fig = bqplot.Figure(marks=[pds_adoption_chart], title='PDS Adoption Basis')
pds_adoption_fig.layout.width = '800px'
pds_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

ref_adoption_chart = bqplot.Pie(sizes=list(ref_adoption_basis_counts.values()),
                                labels=list(ref_adoption_basis_counts.keys()),
                                display_labels='inside', radius=110,
                                display_values=True, values_format='d')
ref_adoption_fig = bqplot.Figure(marks=[ref_adoption_chart], title='REF Adoption Basis')
ref_adoption_fig.layout.width = '300px'
ref_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

ipywidgets.HBox([soln_count_fig, pds_adoption_fig, ref_adoption_fig])